# Automatizando a espacialização dos atendimentos do SP156

Agora que já exploramos os dados, vamos começar a automatizar a espacialização dos dados.

A estratégia vai ser separar por registros que satisfaçam as seguintes condições:

* pontos com Lat/Long
* Distrito e não tenham informação de Setor Fiscal ou Quadra
* Setor fiscal que não tenham informação de Quadra
* Setor e Quadra fiscal

Depois disso podemos agrupar registros por Serviço, Assunto ou Tema conforme a melhor discretização.


In [1]:
import pandas as pd
import glob

path = r'download' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0, encoding="ISO-8859-1", delimiter=';', parse_dates=['Data de abertura', 'Data do parecer'])
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)

# Abrindo o ano de 2019
# df = pd.read_csv('download/dados-do-sp156---1o-sem-2019.csv',index_col=None, header=0, encoding="ISO-8859-1", delimiter=';', parse_dates=['Data de abertura', 'Data do parecer'])
# df.append(pd.read_csv('download/dados-do-sp156---2o-sem-2019.csv',index_col=None, header=0, encoding="ISO-8859-1", delimiter=';', parse_dates=['Data de abertura', 'Data do parecer']))

/home/fernando/miniconda3/envs/geo_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/home/fernando/miniconda3/envs/geo_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/home/fernando/miniconda3/envs/geo_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
df['Latitude'] = pd.to_numeric(df['Latitude'].str.replace(',', '.'), errors='coerce')
df['Longitude'] = pd.to_numeric(df['Longitude'].str.replace(',', '.'), errors='coerce')

In [3]:
df.dtypes

Data de abertura         datetime64[ns]
Canal                            object
Tema                             object
Assunto                          object
Serviço                          object
Logradouro                       object
Número                           object
CEP                              object
Subprefeitura                    object
Distrito                         object
Setor                           float64
Quadra                          float64
Latitude                        float64
Longitude                       float64
Data do parecer          datetime64[ns]
Status da solicitação            object
Órgão                            object
dtype: object

In [4]:
df_points = df[~df.Longitude.isna()]

In [5]:
df_distrito = df[~df.Distrito.isna() & df.Latitude.isna() & df.Setor.isna()]

In [6]:
df_setor = df[df.Latitude.isna() & ~df.Setor.isna() & df.Quadra.isna()]

In [7]:
df_quadra = df[df.Latitude.isna() & ~df.Setor.isna() & ~df.Quadra.isna()]

In [8]:
df_points.Serviço.unique()

array(['Ônibus - Denúncia de direção inadequada ou perigosa de veículo',
       'Tapa-buraco', 'Reclamação sobre iluminação pública (ILUME)',
       'Remoção de entulho em via pública',
       'Tapa-buraco em faixa exclusiva ou corredor de ônibus',
       'Limpeza de bueiros, boca de lobo e poços de visita',
       'Remoção de grandes objetos em vias públicas',
       'Instalação e remoção de lixeira',
       'Reclamação de afundamento da pavimentação em vias públicas',
       'Reclamação de falta de varrição',
       'Limpeza da via pública após enchentes ou eventos',
       'Remoção de animal morto em via pública',
       'Reclamação de não remoção dos sacos de varrição',
       'Ônibus - Reclamação de intervalo excessivo da linha',
       'Cata Bagulho - Reclamação de não retirada de material',
       'Reclamação de má execução do serviço de tapa-buraco',
       'Conserto de lixeira', 'Manutenção de placas de trânsito',
       'Manutenção de sinalização horizontal (pintura de solo)'

In [9]:
df_distrito.Assunto.unique()

array(['População ou pessoa em situação de rua', 'Poluição Sonora - PSIU',
       'Reclamação ônibus',
       'Sinalização e Circulação de Veículos e Pedestres',
       'Capinação e roçada de áreas verdes', 'Iluminação pública',
       'Defesa Civil',
       'Estabelecimentos comerciais, indústrias e serviços',
       'Veículos abandonados', 'Drenagem de água de chuva', 'Ônibus',
       'Árvore', 'Ponto viciado, entulho e caçamba de entulho',
       'Terrenos e imóveis', 'Ambulantes', 'Fiscalização de obras',
       'Animal em via pública', 'Reparos em asfalto, pontes e viadutos',
       'Rios e Córregos', 'Calçadas, guias e postes',
       'Dengue/chikungunya/zika (mosquito aedes aegypti)',
       'Coleta de lixo domiciliar',
       'Animais que podem causar doenças e/ou agravos à saúde',
       'Áreas municipais', 'Guarda Civil Metropolitana',
       'Ruas, vilas, vielas e escadarias', 'Praças',
       'Criação inadequada de animais', 'Esgoto e água usada',
       'Estacionamento e Z

In [10]:
df_setor.Serviço.unique()

array(['Reclamações sobre coleta de lixo domiciliar',
       'Reclamações sobre coleta seletiva',
       'Denúncia de má conduta dos funcionários (motoristas /coletores)',
       'Reclamações sobre coleta de resíduos de serviços de saúde',
       'Remoção de propaganda irregular',
       'Denúncia de má conduta dos funcionários de limpeza'], dtype=object)

In [11]:
df_quadra.Serviço.unique()

array(['Remover animal de grande porte solto',
       'Vistoria de colmeia/vespeiro instalado',
       'Avaliação de animal em sofrimento sem proprietário para eutanásia (morte sem dor)',
       'Avaliação de local com pernilongo/mosquito',
       'Reclamação de local com morcego',
       'Avaliação de animal agressor em vias públicas',
       'Animal encontrado com registro geral do animal (RGA)',
       'Implantação de coleta porta-porta'], dtype=object)

In [12]:
# TODO
# Separar por ano
# FAzer um layer com todos os Distritos concentrados

In [13]:
df.Tema.unique()

array(['Cidadania e assistência social', 'Rua e bairro',
       'Trânsito e Transporte', 'Animais', 'Urgência',
       'Comércio e serviços', 'Lixo e limpeza', 'Segurança urbana',
       'Obras, imóveis e habitação', 'Meio Ambiente', 'Saúde',
       'Canais de Atendimento', 'Trabalho',
       'Ouvidoria Geral do Município', 'Bilhete Único', 'SAC/GRC',
       'Educação', 'Cultura, esporte e lazer', 'Acessibilidade',
       'Documentos e licenças', 'Finanças', nan, 'Descomplica São Miguel',
       'Serviço funerário'], dtype=object)

## Separando por ano

In [14]:
df['Data de abertura'].groupby([df['Data de abertura'].dt.year]).count()

Data de abertura
2015.0     752915
2016.0     778690
2017.0     732429
2018.0     747606
2019.0    1126623
2020.0     349230
Name: Data de abertura, dtype: int64

In [15]:
anos = df[~df['Data de abertura'].isna()]['Data de abertura'].dt.year.unique()
print(anos)

[2018 2015 2017 2019 2016 2020]


## Pontos

No caso dos pontos podemos gerar um arquivo sem discretização, ou seja, cada registro conterá seu ponto.

Vamos utilizar uma estratégia para agrupa-los por `Serviço` dessa forma:

In [16]:
len(df_points.Serviço.unique())

42

In [17]:
import geopandas as gpd

In [18]:
gdf_points = gpd.GeoDataFrame(df_points, geometry=gpd.points_from_xy(df_points.Longitude, df_points.Latitude))

In [19]:
gdf_points.crs = 'epsg:4989'
gdf_points.to_crs('epsg:31983')

,Data de abertura,Canal,Tema,Assunto,Serviço,Logradouro,Número,CEP,Subprefeitura,Distrito,Setor,Quadra,Latitude,Longitude,Data do parecer,Status da solicitação,Órgão,geometry
8,2018-01-01 00:55:54,PORTAL ONLINE,Trânsito e Transporte,Reclamação ônibus,Ônibus - Denúncia de direção inadequada ou per...,Avenida Ragueb Chohfi,2618,08375000,CIDADE TIRADENTES,CIDADE TIRADENTES,237.0,16.0,-23.585249,-46.414783,2020-02-26 13:24:01,AGUARDANDO ATENDIMENTO,SPTRANS,POINT (355633.495 7390977.687)
13,2018-01-01 01:48:59,INTEGRAÇÕES,Rua e bairro,"Reparos em asfalto, pontes e viadutos",Tapa-buraco,"INTERLAGOS, AV",0,04660000,CAPELA DO SOCORRO,CIDADE DUTRA,NaN,NaN,-23.688044,-46.696688,2018-01-09 11:00:53,FINALIZADA,SMSUB,POINT (326997.347 7379281.343)
14,2018-01-01 01:49:03,INTEGRAÇÕES,Rua e bairro,"Reparos em asfalto, pontes e viadutos",Tapa-buraco,"INTERLAGOS, R",5417,04679000,SANTO AMARO,CAMPO GRANDE,NaN,NaN,-23.676890,-46.681698,2018-06-18 10:39:10,FINALIZADA,SMSUB,POINT (328511.546 7380534.638)
23,2018-01-01 03:06:02,CENTRAL TELEFÔNICA,Rua e bairro,"Reparos em asfalto, pontes e viadutos",Tapa-buraco,R ANTONIO CARNEIRO LEAO,290,03548080,PENHA,VILA MATILDE,113.0,77.0,-23.542519,-46.508059,2018-04-09 17:05:56,INDEFERIDO,SMSUB,POINT (346064.145 7395612.119)
26,2018-01-01 03:11:51,CENTRAL TELEFÔNICA,Rua e bairro,"Reparos em asfalto, pontes e viadutos",Tapa-buraco,AV LUIZ DUMONT VILLARES,340,02085100,VILA MARIA-VILA GUILHERME,VILA GUILHERME,68.0,560.0,-23.499242,-46.613323,2018-01-04 15:37:56,FINALIZADA,SMSUB,POINT (335263.541 7400287.570)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4487469,2020-03-31 21:34:00,PORTAL ONLINE,Rua e bairro,"Reparos em asfalto, pontes e viadutos",Tapa-buraco em faixa exclusiva ou corredor de ...,Avenida Senador Teotônio Vilela,598,4.801e+06,CAPELA DO SOCORRO,CIDADE DUTRA,95.0,84.0,-23.707440,-46.700294,2020-03-31 21:35:00,AGUARDANDO ATENDIMENTO,SPTRANS,POINT (326655.277 7377128.929)
4487483,2020-03-31 21:40:00,PORTAL ONLINE,Lixo e limpeza,"Ponto viciado, entulho e caçamba de entulho",Remoção de entulho em via pública,Rua dos Pica Paus,115,4.32308e+06,JABAQUARA,JABAQUARA,NaN,NaN,-23.653971,-46.640840,NaT,AGUARDANDO ATENDIMENTO,AMLURB,POINT (332649.548 7383121.192)
4487491,2020-03-31 21:44:00,APLICATIVO,Rua e bairro,"Reparos em asfalto, pontes e viadutos",Tapa-buraco,Avenida Jandira,850,4.08000e+06,VILA MARIANA,MOEMA,45.0,169.0,-23.610862,-46.658034,2020-03-31 21:44:00,AGUARDANDO ATENDIMENTO,SMSUB,POINT (330840.182 7387874.854)
4487493,2020-03-31 21:45:00,CENTRAL TELEFÔNICA,Trânsito e Transporte,Reclamação ônibus,Ônibus - Reclamação de intervalo excessivo da ...,Avenida Brigadeiro Faria Lima,4300,4.54504e+06,PINHEIROS,ITAIM BIBI,299.0,51.0,-23.594309,-46.681137,2020-03-31 21:46:00,AGUARDANDO ATENDIMENTO,SPTRANS,POINT (328461.144 7389680.480)


In [20]:
for y in anos:
    for s in df_points[df_points['Data de abertura'].dt.year == y].Serviço.unique():
        print(f"{s} ({y}): ({len(df_points[df_points.Serviço == s])})")
        (gdf_points[(df_points.Serviço == s) & (df_points['Data de abertura'].dt.year == y)]
         .to_file(f'resultados/Atendimentos-156-{y}.gpkg', layer=f"Pontos - {s}", driver='GPKG'))

Ônibus - Denúncia de direção inadequada ou perigosa de veículo (2018): (25339)
Tapa-buraco (2018): (549708)
Reclamação sobre iluminação pública (ILUME) (2018): (2975)
Remoção de entulho em via pública (2018): (95774)
Tapa-buraco em faixa exclusiva ou corredor de ônibus (2018): (8474)
Limpeza de bueiros, boca de lobo e poços de visita (2018): (84383)
Remoção de grandes objetos em vias públicas (2018): (110158)
Instalação e remoção de lixeira (2018): (22053)
Reclamação de afundamento da pavimentação em vias públicas (2018): (14582)
Reclamação de falta de varrição (2018): (84556)
Limpeza da via pública após enchentes ou eventos (2018): (8748)
Remoção de animal morto em via pública (2018): (18813)
Reclamação de não remoção dos sacos de varrição (2018): (6113)
Ônibus - Reclamação de intervalo excessivo da linha (2018): (68211)
Cata Bagulho - Reclamação de não retirada de material (2018): (16731)
Reclamação de má execução do serviço de tapa-buraco (2018): (4338)
Conserto de lixeira (2018): (

Reclamação de falta de lavagem de calçadões (2019): (371)
Remoção de contêiner para coleta de lixo domiciliar (2019): (1555)
Manutenção de sinalização horizontal (pintura de solo) (2019): (14947)
Instalação de contêiner para coleta seletiva (2019): (1864)
Selo de identificação de estabelecimento gerador de resíduos de saúde  1ª e 2ª via (2019): (710)
Remoção do contêiner de coleta seletiva (2019): (319)
Proposta para implantação de ciclovias, ciclofaixas e ciclorrotas (2019): (626)
Manutenção de pintura e sinalização de ciclovias, ciclofaixas e ciclorrotas (2019): (561)
Avaliação da infraestrutura de ciclovias, ciclofaixas e ciclorrotas existentes (2019): (414)
Limpeza e conservação de monumentos públicos (2019): (488)
Avaliação de ciclofaixas de lazer (2019): (49)
Manutenção de contêiner de resíduos do serviço de saúde (2019): (81)
Manutenção de caminhão de lixo (2019): (6113)
Roubo do contêiner de coleta seletiva (2019): (13)
Fiscalização de veículo estacionado em local proibido (20

## Agrupando total de atendimentos por distrito

Parece fazer sentido ter um agrupamento geral discretizado por distrito. A princípio separado por número de atendimentos e tema. Para isso vamos ter que relacionar os registros de atendimento às feiçõe de distrito.

Porém precisamos agrupar os atendimentos por distrito.

In [21]:
len(df.Distrito.unique())

97

In [22]:
df['Ano'] = df['Data de abertura'].dt.year

In [23]:
##TODO
# Verificar se existe diversidade de Canais

In [24]:
total_de_atendimentos_por_distrito = (df[['Distrito', 'Ano', 'Canal', 'Tema']]
                                      .groupby(['Distrito', 'Ano'])
                                      .size())

In [25]:
total_de_atendimentos_por_distrito

Distrito    Ano   
AGUA RASA   2015.0    6279
            2016.0    6319
            2017.0    6145
            2018.0    7015
            2019.0    9145
                      ... 
VILA SONIA  2016.0    8053
            2017.0    7998
            2018.0    6223
            2019.0    8325
            2020.0    2218
Length: 576, dtype: int64

In [26]:
canais_e_temas_por_distrito = (pd.get_dummies(df[['Distrito', 'Ano', 'Canal', 'Tema']][~df['Distrito'].isna()], columns=['Canal', 'Tema'])
                               .groupby(['Distrito', 'Ano'])
                               .sum())

In [27]:
df_distritos_e_total_por_distrito = pd.concat([total_de_atendimentos_por_distrito.to_frame(name='Total de atendimentos'), canais_e_temas_por_distrito], axis=1)
# total_de_atendimentos_por_distrito.to_frame()

In [28]:
df_distritos = gpd.read_file('arquivos/SIRGAS_SHP_distrito_polygon.shp')

In [29]:
df_distritos_e_total_por_distrito = df_distritos.merge(df_distritos_e_total_por_distrito, left_on='ds_nome', right_on='Distrito')

In [30]:
df_distritos_e_total_por_distrito[canais_e_temas_por_distrito.index.get_level_values(1) == 2015]

,ds_codigo,ds_nome,geometry,Total de atendimentos,Canal_,Canal_APLICATIVO,Canal_CENTRAL TELEFÔNICA,Canal_DESCOMPLICA BUTANTÃ,Canal_DESCOMPLICA CAMPO LIMPO,Canal_DESCOMPLICA JABAQUARA,...,Tema_Meio Ambiente,"Tema_Obras, imóveis e habitação",Tema_Ouvidoria Geral do Município,Tema_Rua e bairro,Tema_SAC/GRC,Tema_Saúde,Tema_Segurança urbana,Tema_Trabalho,Tema_Trânsito e Transporte,Tema_Urgência
0,51,MANDAQUI,"POLYGON ((330950.373 7407837.176, 330952.177 7...",6062,6062.0,0.0,0.0,0.0,0.0,0.0,...,38.0,177.0,0.0,2473.0,2.0,450.0,2.0,1.0,1282.0,0.0
6,52,MARSILAC,"POLYGON ((336124.090 7355302.282, 336121.755 7...",118,118.0,0.0,0.0,0.0,0.0,0.0,...,1.0,2.0,0.0,43.0,1.0,0.0,0.0,0.0,45.0,0.0
12,32,MOEMA,"POLYGON ((331242.174 7392162.324, 331244.581 7...",5872,5872.0,0.0,0.0,0.0,0.0,0.0,...,38.0,218.0,0.0,2728.0,7.0,360.0,19.0,0.0,1353.0,0.0
18,57,PARQUE DO CARMO,"POLYGON ((352597.619 7390119.444, 352588.233 7...",4071,4071.0,0.0,0.0,0.0,0.0,0.0,...,32.0,109.0,0.0,2012.0,4.0,264.0,13.0,0.0,541.0,0.0
24,60,PERDIZES,"POLYGON ((330181.745 7396056.840, 330175.535 7...",6845,6845.0,0.0,0.0,0.0,0.0,0.0,...,8.0,236.0,0.0,2971.0,5.0,519.0,5.0,1.0,1136.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,41,JAGUARE,"POLYGON ((322952.516 7394978.618, 323250.620 7...",2927,2927.0,0.0,0.0,0.0,0.0,0.0,...,4.0,115.0,0.0,1155.0,5.0,146.0,5.0,0.0,588.0,0.0
552,42,JARAGUA,"POLYGON ((319172.403 7405179.204, 319174.204 7...",6525,6525.0,0.0,0.0,0.0,0.0,0.0,...,62.0,149.0,0.0,2852.0,10.0,435.0,9.0,0.0,1342.0,1.0
558,44,JARDIM HELENA,"POLYGON ((352915.621 7402621.049, 352944.248 7...",3793,3793.0,0.0,0.0,0.0,0.0,0.0,...,40.0,75.0,0.0,1400.0,6.0,262.0,4.0,0.0,478.0,0.0
564,45,JARDIM PAULISTA,"POLYGON ((329605.461 7390837.489, 329600.764 7...",6817,6817.0,0.0,0.0,0.0,0.0,0.0,...,16.0,213.0,0.0,4117.0,9.0,300.0,2.0,0.0,920.0,0.0


In [31]:
for y in anos:
    (df_distritos_e_total_por_distrito[canais_e_temas_por_distrito.index.get_level_values(1) == y]
     .to_file(f'resultados/Atendimentos-156-{y}.gpkg', layer='Distritos - Quadro geral por canal e tema', driver='GPKG'))

## Assuntos por distrito

Agora podemos começar a gerar as uma camada para cada Assunto discretizado por distrito. Ou seja, vamos separar por assuntos os registros de atendimentos que possuem somente informação do distrito

In [34]:
df_distrito[df_distrito['Data de abertura'].dt.year == 2019]

,Data de abertura,Canal,Tema,Assunto,Serviço,Logradouro,Número,CEP,Subprefeitura,Distrito,Setor,Quadra,Latitude,Longitude,Data do parecer,Status da solicitação,Órgão
1062835,2019-01-01 00:29:43,CENTRAL TELEFÔNICA,Cidadania e assistência social,População ou pessoa em situação de rua,Serviço Especializado de Abordagem Social às P...,NaN,NaN,NaN,BUTANTA,VILA SONIA,NaN,NaN,NaN,NaN,2019-01-01 04:44:28,FINALIZADA,SMADS
1062836,2019-01-01 00:32:04,CENTRAL TELEFÔNICA,Cidadania e assistência social,População ou pessoa em situação de rua,Serviço Especializado de Abordagem Social às P...,NaN,NaN,NaN,CAMPO LIMPO,CAMPO LIMPO,NaN,NaN,NaN,NaN,2019-01-01 04:44:28,FINALIZADA,SMADS
1062837,2019-01-01 00:35:47,CENTRAL TELEFÔNICA,Cidadania e assistência social,População ou pessoa em situação de rua,Serviço Especializado de Abordagem Social às P...,NaN,NaN,NaN,CAMPO LIMPO,CAMPO LIMPO,NaN,NaN,NaN,NaN,2019-01-01 04:44:28,FINALIZADA,SMADS
1062838,2019-01-01 00:38:05,CENTRAL TELEFÔNICA,Cidadania e assistência social,População ou pessoa em situação de rua,Serviço Especializado de Abordagem Social às P...,NaN,NaN,NaN,ERMELINO MATARAZZO,ERMELINO MATARAZZO,NaN,NaN,NaN,NaN,2019-01-01 05:30:50,FINALIZADA,SMADS
1062839,2019-01-01 00:43:45,CENTRAL TELEFÔNICA,Cidadania e assistência social,População ou pessoa em situação de rua,Serviço Especializado de Abordagem Social às P...,NaN,NaN,NaN,MOOCA,BRAS,NaN,NaN,NaN,NaN,2019-01-01 04:44:28,FINALIZADA,SMADS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2879915,2019-12-31 20:43:00,CENTRAL TELEFÔNICA,Cidadania e assistência social,População ou pessoa em situação de rua,Serviço Especializado de Abordagem Social às P...,NaN,NaN,NaN,SE,CAMBUCI,NaN,NaN,NaN,NaN,2019-12-31 21:03:00,FINALIZADA,SMADS
2879916,2019-12-31 20:47:00,CENTRAL TELEFÔNICA,Cidadania e assistência social,População ou pessoa em situação de rua,Serviço Especializado de Abordagem Social às P...,NaN,NaN,NaN,SE,SANTA CECILIA,NaN,NaN,NaN,NaN,2019-12-31 21:06:00,FINALIZADA,SMADS
2879922,2019-12-31 21:06:00,CENTRAL TELEFÔNICA,Cidadania e assistência social,População ou pessoa em situação de rua,Serviço Especializado de Abordagem Social às P...,NaN,NaN,NaN,LAPA,BARRA FUNDA,NaN,NaN,NaN,NaN,2019-12-31 21:47:00,FINALIZADA,SMADS
2879923,2019-12-31 21:06:00,CENTRAL TELEFÔNICA,Cidadania e assistência social,População ou pessoa em situação de rua,Serviço Especializado de Abordagem Social às P...,NaN,NaN,NaN,SANTANA-TUCURUVI,SANTANA,NaN,NaN,NaN,NaN,2019-12-31 21:46:00,FINALIZADA,SMADS


In [42]:
for y in anos:
    for a in df_distrito.Assunto.unique():
        print(f"{a} ({y}): ({len(df_distrito[(df_distrito.Assunto == a) & (df_distrito['Data de abertura'].dt.year == y)])})")
        c = (df_distrito[(df_distrito.Assunto == a) & (df_distrito['Data de abertura'].dt.year == y)][['Distrito','Data de abertura']]
             .groupby('Distrito')
             .count()
             .rename(columns={'Data de abertura':'Total de Atendimentos'}))
        r = df_distritos.merge(c, left_on='ds_nome', right_on='Distrito')
        if len(r) > 1:
            r.to_file(f'resultados/Atendimentos-156-{y}.gpkg', layer=f'Distritos por assunto - {a}', driver='GPKG')

População ou pessoa em situação de rua (2018): (91610)
Poluição Sonora - PSIU (2018): (22166)
Reclamação ônibus (2018): (30069)
Sinalização e Circulação de Veículos e Pedestres (2018): (7107)
Capinação e roçada de áreas verdes (2018): (21996)
Iluminação pública (2018): (1189)
Defesa Civil (2018): (18739)
Estabelecimentos comerciais, indústrias e serviços (2018): (9271)
Veículos abandonados (2018): (28679)
Drenagem de água de chuva (2018): (17775)
Ônibus (2018): (10512)
Árvore (2018): (54236)
Ponto viciado, entulho e caçamba de entulho (2018): (9136)
Terrenos e imóveis (2018): (8984)
Ambulantes (2018): (3081)
Fiscalização de obras (2018): (13638)
Animal em via pública (2018): (638)
Reparos em asfalto, pontes e viadutos (2018): (1206)
Rios e Córregos (2018): (3895)
Calçadas, guias e postes (2018): (15328)
Dengue/chikungunya/zika (mosquito aedes aegypti) (2018): (5793)
Coleta de lixo domiciliar (2018): (2630)
Animais que podem causar doenças e/ou agravos à saúde (2018): (10692)
Áreas muni

Imigrante (2015): (0)
Centro de Formação Cultural da Cidade Tiradentes (2015): (0)
Programa Ação Jovem (2015): (33)
Transporte Escolar (2015): (0)
Água Subterrânea/Curso d'Água (2015): (50)
ATENDE - Transporte Pessoas com Deficiência (2015): (162)
LGBTI (2015): (0)
Comunicação (2015): (0)
Áreas Contaminadas (2015): (0)
Programa Operação Trabalho - POT nas Subprefeituras (2015): (0)
Crimes contra a vida (2015): (0)
Programa Renda Cidadã (2015): (82)
Outras reclamações e denúncias (2015): (0)
Drogas (2015): (0)
Programa de Erradicação do Trabalho Infantil (PETI) (2015): (3)
Central 156 (2015): (1921)
Mulher (2015): (0)
Questões raciais, étnicas e religiosas (2015): (6)
Jovem (2015): (1)
Remoção de grandes objetos (2015): (25685)
Varrição e limpeza urbana (2015): (4030)
Entulho e caçambas de entulho (2015): (1691)
Apresentar sinais de embriaguez em serviço (2015): (205)
BU Comum (2015): (1099)
Lixão clandestino (2015): (1192)
PROGRAMA NOTA FISCAL PAULISTANA - N.F. DE SERVIÇOS ELETRÔNICA -

Veículos abandonados (2019): (30148)
Drenagem de água de chuva (2019): (16066)
Ônibus (2019): (12188)
Árvore (2019): (65096)
Ponto viciado, entulho e caçamba de entulho (2019): (10080)
Terrenos e imóveis (2019): (10894)
Ambulantes (2019): (3619)
Fiscalização de obras (2019): (14844)
Animal em via pública (2019): (473)
Reparos em asfalto, pontes e viadutos (2019): (1762)
Rios e Córregos (2019): (3967)
Calçadas, guias e postes (2019): (18042)
Dengue/chikungunya/zika (mosquito aedes aegypti) (2019): (10850)
Coleta de lixo domiciliar (2019): (2549)
Animais que podem causar doenças e/ou agravos à saúde (2019): (11735)
Áreas municipais (2019): (3501)
Guarda Civil Metropolitana (2019): (14875)
Ruas, vilas, vielas e escadarias (2019): (1449)
Praças (2019): (3248)
Criação inadequada de animais (2019): (6101)
Esgoto e água usada (2019): (2979)
Estacionamento e Zona Azul (2019): (5438)
Reciclagem (2019): (5005)
Mercados e sacolões (2019): (10)
Ouvidoria Geral do Município (2019): (24783)
Comida d

Crimes contra a vida (2016): (1)
Programa Renda Cidadã (2016): (41)
Outras reclamações e denúncias (2016): (0)
Drogas (2016): (0)
Programa de Erradicação do Trabalho Infantil (PETI) (2016): (2)
Central 156 (2016): (1371)
Mulher (2016): (0)
Questões raciais, étnicas e religiosas (2016): (3)
Jovem (2016): (0)
Remoção de grandes objetos (2016): (27702)
Varrição e limpeza urbana (2016): (3748)
Entulho e caçambas de entulho (2016): (1594)
Apresentar sinais de embriaguez em serviço (2016): (147)
BU Comum (2016): (3707)
Lixão clandestino (2016): (952)
PROGRAMA NOTA FISCAL PAULISTANA - N.F. DE SERVIÇOS ELETRÔNICA - NFS-E (2016): (45)
Leve Leite (2016): (164)
NF Paulistana (Comunicação de não conversão  RPS) (2016): (23)
Alimentação escolar (2016): (54)
Programa Renda Mínima (2016): (133)
Fio faiscando (2016): (462)
Obras na via (2016): (55)
IPTU (2016): (132)
Lavar/Reparar veiculo em via publica (2016): (49)
Valets e estacionamentos particulares (2016): (91)
Ouvidoria da saúde (2016): (46)
Co

## Serviços por Setor Fiscal e Setor e Quadra fiscal

Analogamente o que fizemos com os distritos, agora vamos relacionar os registros que tenham apenas Setor fiscal, assim com Setor e Quadra fiscal, discretizando os registros nas feições.

In [43]:
zipfile = "zip:///home/fernando/dev/geo-sp156/arquivos/SIRGAS_SHP_setorfiscal.zip!SIRGAS_SHP_setorfiscal/SIRGAS_SHP_setorfiscal.shp"
df_setores = gpd.read_file(zipfile)

In [44]:
# df_setores

In [45]:
df_setor['st_codigo'] = df_setor.Setor.astype(int).apply(lambda x: '{:03d}'.format(x))

<ipython-input-45-b4f69922e5aa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_setor['st_codigo'] = df_setor.Setor.astype(int).apply(lambda x: '{:03d}'.format(x))


In [46]:
# df_setor

In [49]:
for y in anos:
    for s in df_setor.Serviço.unique():
        print(f"{s} ({y}): ({len(df_setor[(df_setor.Serviço == s) & (df_setor['Data de abertura'].dt.year == y)])})")
        c = (df_setor[(df_setor.Serviço == s) & (df_setor['Data de abertura'].dt.year == y)][['st_codigo','Data de abertura']]
             .groupby('st_codigo')
             .count()
             .rename(columns={'Data de abertura':'Total de Atendimentos'}))
        r = df_setores.merge(c, on='st_codigo')
        if len(r) > 1:
            r.to_file(f'resultados/Atendimentos-156-{y}.gpkg', layer=f'Setores por serviço - {s}', driver='GPKG')

Reclamações sobre coleta de lixo domiciliar (2018): (4848)
Reclamações sobre coleta seletiva (2018): (1146)
Denúncia de má conduta dos funcionários (motoristas /coletores) (2018): (681)
Reclamações sobre coleta de resíduos de serviços de saúde (2018): (436)
Remoção de propaganda irregular (2018): (214)
Denúncia de má conduta dos funcionários de limpeza (2018): (381)
Reclamações sobre coleta de lixo domiciliar (2015): (2997)
Reclamações sobre coleta seletiva (2015): (1438)
Denúncia de má conduta dos funcionários (motoristas /coletores) (2015): (1306)
Reclamações sobre coleta de resíduos de serviços de saúde (2015): (192)
Remoção de propaganda irregular (2015): (621)
Denúncia de má conduta dos funcionários de limpeza (2015): (242)
Reclamações sobre coleta de lixo domiciliar (2017): (4000)
Reclamações sobre coleta seletiva (2017): (594)
Denúncia de má conduta dos funcionários (motoristas /coletores) (2017): (557)
Reclamações sobre coleta de resíduos de serviços de saúde (2017): (245)
Remo

In [50]:
zipfile = "zip:///home/fernando/dev/geo-sp156/arquivos/SIRGAS_SHP_quadraMDSF.zip!SIRGAS_SHP_quadraMDSF/SIRGAS_SHP_quadraMDSF.shp"
df_quadras = gpd.read_file(zipfile)

In [51]:
df_quadras.head(1)

,qd_id_orig,qd_setor,qd_fiscal,qd_tipo,qd_subqua,qd_area,qd_situac,qd_id,qd_tx_tipo,geometry
0,10346990,153,029,F,001,4859.1528512042,1,129531,FISCAL,"POLYGON ((347209.277 7387175.531, 347202.470 7..."


In [52]:
df_quadra['qd_setor'] = df_quadra.Setor.astype(int).apply(lambda x: '{:03d}'.format(x))
df_quadra['qd_fiscal'] = df_quadra.Quadra.astype(int).apply(lambda x: '{:03d}'.format(x))

<ipython-input-52-c582b7a93ab0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_quadra['qd_setor'] = df_quadra.Setor.astype(int).apply(lambda x: '{:03d}'.format(x))
<ipython-input-52-c582b7a93ab0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_quadra['qd_fiscal'] = df_quadra.Quadra.astype(int).apply(lambda x: '{:03d}'.format(x))


In [53]:
df_quadra.head(1)

,Data de abertura,Canal,Tema,Assunto,Serviço,Logradouro,Número,CEP,Subprefeitura,Distrito,Setor,Quadra,Latitude,Longitude,Data do parecer,Status da solicitação,Órgão,qd_setor,qd_fiscal
15,2018-01-01 01:50:49,CENTRAL TELEFÔNICA,Animais,Animal em via pública,Remover animal de grande porte solto,AV GIOVANNI GRONCHI,NaN,NaN,BUTANTA,MORUMBI,123.0,128.0,NaN,NaN,2018-01-02 11:59:26,FINALIZADA,COVISA-DVZ,123,128


In [55]:
for y in anos:
    for s in df_quadra.Serviço.unique():
        print(f"{s} ({y}): ({len(df_quadra[(df_quadra.Serviço == s) & (df_quadra['Data de abertura'].dt.year == y)])})")
        c = (df_quadra[(df_quadra.Serviço == s) & (df_quadra['Data de abertura'].dt.year == y)][['qd_setor', 'qd_fiscal','Data de abertura']]
             .groupby(['qd_setor', 'qd_fiscal'])
             .count()
             .rename(columns={'Data de abertura':'Total de Atendimentos'}))
        r = df_quadras.merge(c, on=['qd_setor', 'qd_fiscal'])
        if len(r) > 1:
            r.to_file(f'resultados/Atendimentos-156-{y}.gpkg', layer=f'Quadras por serviço - {s}', driver='GPKG')

Remover animal de grande porte solto (2018): (242)
Vistoria de colmeia/vespeiro instalado (2018): (8274)
Avaliação de animal em sofrimento sem proprietário para eutanásia (morte sem dor) (2018): (1170)
Avaliação de local com pernilongo/mosquito (2018): (2898)
Reclamação de local com morcego (2018): (817)
Avaliação de animal agressor em vias públicas (2018): (994)
Animal encontrado com registro geral do animal (RGA) (2018): (88)
Implantação de coleta porta-porta (2018): (339)
Remover animal de grande porte solto (2015): (156)
Vistoria de colmeia/vespeiro instalado (2015): (8792)
Avaliação de animal em sofrimento sem proprietário para eutanásia (morte sem dor) (2015): (2676)
Avaliação de local com pernilongo/mosquito (2015): (3583)
Reclamação de local com morcego (2015): (582)
Avaliação de animal agressor em vias públicas (2015): (4394)
Animal encontrado com registro geral do animal (RGA) (2015): (65)
Implantação de coleta porta-porta (2015): (205)
Remover animal de grande porte solto (2

## TODO Trantando pontos fora dos limites do município de São Paulo

Existem casos em que os pontos estão localizados fora dos limites do município de São Paulo e para finalidade de estudo vamos quantifica-los e discriminá-los para estudos posteriores.

In [118]:
# msp = geopandas.read_file('arquivos/SP.shp')

In [169]:
# gdf_sp = geopandas.sjoin(gdf, msp, how='left', op='within')

In [170]:
# gdf_sp[gdf_sp.municipio.isna()].to_file("resultados/fora_dos_limites.gpkg", layer='fora_dos_limites', driver="GPKG")